In [2]:
import pandas as pd
import matplotlib.pyplot
import datetime as dt

df = pd.read_csv('money.csv',sep=',')
df['order_date'] = pd.to_datetime(df['order_date'])
df.head()

,order_date,order_id,customer,grand_total
0,2010-12-01,1,United Kingdom,4.91
1,2010-12-05,1,United Kingdom,29.46
2,2010-12-20,1,United Kingdom,4.91
3,2011-01-05,1,United Kingdom,2.46
4,2011-01-26,1,United Kingdom,2.46


In [3]:
NOW = dt.datetime(2011,12,31)

In [4]:
rfmTable = df.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()).days, # Recency
                                        'order_id': lambda x: len(x),
                                        'grand_total': lambda x: x.sum()}) # Monetary Value
rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date':'recency','order_id':'frequency','grand_total': 'monetary_value'}, inplace=True)
rfmTable.head()

,recency,frequency,monetary_value
customer,,,
Australia,37,1182,187980.48
Austria,23,398,11870.89
Bahrain,226,18,845.64
Belgium,22,2031,53120.26
Brazil,260,32,1487.42


In [5]:
def pct_rank_qcut(series, n):
    edges = pd.Series([float(i) / n for i in range(n + 1)])
    f = lambda x: (edges >= x).argmax()
    return series.rank(pct=1).apply(f)

In [6]:
# 最近消費日期
rfmTable['rec_dec'] = pct_rank_qcut(rfmTable['recency'], 5)

In [7]:
# 頻率
freq_dec = pct_rank_qcut(rfmTable['frequency'], 5)
rfmTable['freq_dec'] = 6 - freq_dec

In [8]:
# 消費金額
mv_dec = pct_rank_qcut(rfmTable['monetary_value'], 5)
rfmTable['mv_dec'] = 6 - mv_dec

In [9]:
rfmTable['RFMClass'] = rfmTable.rec_dec.map(str) + rfmTable.freq_dec.map(str) + rfmTable.mv_dec.map(str)
rfmTable

,recency,frequency,monetary_value,rec_dec,freq_dec,mv_dec,RFMClass
customer,,,,,,,
Australia,37,1182,1.879805e+05,3,2,1,321
Austria,23,398,1.187089e+04,1,3,3,133
Bahrain,226,18,8.456400e+02,5,5,5,555
Belgium,22,2031,5.312026e+04,1,1,2,112
Brazil,260,32,1.487420e+03,5,5,5,555
Canada,128,151,6.023920e+03,5,4,4,544
Channel Islands,26,748,2.472476e+04,3,2,2,322
Cyprus,43,614,1.613793e+04,4,2,3,423
Czech Republic,85,25,1.000700e+03,4,5,5,455


In [10]:
rfmTable.to_csv("sss.csv",columns=['recency','frequency', 'monetary_value', 'rec_dec','freq_dec','mv_dec','RFMClass'])

In [13]:
!pip install matplotlib

In [14]:
import matplotlib.pyplot as plt

In [15]:
rec_dec_m = rfmTable.groupby(['rec_dec'])['monetary_value'].mean()
rec_dec_m.plot(kind='bar')

In [46]:
rec_dec_m = rfmTable.groupby(['rec_dec'])['monetary_value'].mean()
rec_dec_m.plot(kind='bar')